<a href="https://colab.research.google.com/github/shubhka/Automatic-Vector-Digitization/blob/main/BuildingExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print('Good Night World')

Good Night World
